In [1]:
import sys
import matplotlib.pyplot as plt


#sys.path.append("../C_20_data")
#sys.path.append("../thesis-carbon-folding/Unfolding")
#sys.path.append("../thesis-carbon-folding/functions")
#sys.path.append("../thesis-carbon-folding/Unfolding/Application_functions")
#sys.path.append("../data/")
sys.path.append("../unfolding/")

from Application_functions import read_unfolding
from read_geometries import read_geometry
from geometry_functions import *

dual_unfolding, graph_unfolding, graph_unfolding_faces, vertices_final, bonds_toBe, lengths_toBe, angles_f, opt_geom, halogen_positions, neighbours, graph_faces = read_unfolding("../data/unfolding_graph/C60data.h5")
#halogene_positions = np.array([20, 27, 34])
from Unfolding import Unfolding
from functions_folding import *
from interpolation import interpolated_values
import pandas as pd
import h5py

f = open("../data/header/header_cam-b3lyp_6-31star", "r")
header_cam_b3lyp = f.read()
f.close()

In [2]:
header_cam_b3lyp

'%nprocshared=64\n%mem=100GB\n# Opt=ReadFreeze cam-b3lyp/6-31g* empiricaldispersion=gd3\n\nPrecursor Fragment\n\n0 1\n'

In [3]:
def three_dist(unfolding):
    dist_a, dist_b, dist_c = np.linalg.norm(C60.vertex_coords[15] - C60.vertex_coords[35]) , np.linalg.norm(C60.vertex_coords[21] - C60.vertex_coords[22]), np.linalg.norm(C60.vertex_coords[28] - C60.vertex_coords[29])  
    mean_dist = np.mean([dist_a,dist_b,dist_c])
    return np.round(mean_dist,3)

In [4]:
def sample_unfolding(unfolding, title_body="../C60_hinge_0_B3LYP", header=header_cam_b3lyp , start=1.8, end=1.5, step = 0.05, epsilon=0.001,halogen=0):
    sample_r = np.flip(np.arange(end, start, step))
    ### Set the atoms that should be frozen ###
    freeze_list = [15,21,22,28,29,35]
    ### Set the carbon atoms where the halogens should be placed ###
    atoms = [35, 15, 21, 22, 28, 29, 51, 19, 55, 26, 59, 33, 34, 36, 20, 40, 27, 44]
    dist = three_dist(unfolding)
    title = "../" + title_body + "_" + str(dist) +"A"
    write_gaussfile_restart(unfolding, header, freeze=True, connectivity=False, writeFile=True, rotate=True, atoms=atoms, filename = title , freezelist = freeze_list, halogen=halogen, interpolated_angles=interpolated_values)
    for _ in range(unfolding.num_of_steps):
        unfolding.close_unfolding()
        dist = three_dist(unfolding)
        if np.linalg.norm(dist - sample_r[0]) < epsilon:
            sample_r = np.delete(sample_r,0)
            title = title_body + "_" + str(dist) +"A"
            write_gaussfile_restart(unfolding, header, freeze=True, connectivity=False, writeFile=True, rotate=True, atoms=atoms, filename = title , freezelist = freeze_list, halogen=halogen, interpolated_angles=interpolated_values)
            if len(sample_r) == 0:
                return

In [5]:
### Init unfolding and set the number of steps in which the angles are to be closed ###
C60 = Unfolding(dual_unfolding, graph_unfolding_faces, graph_faces, graph_unfolding, neighbours, halogen_positions=halogen_positions, root_node=0, bonds_toBe=bonds_toBe, angles_f=angles_f)
C60.num_of_steps = 20000

root_node = 0, root_face = [0, 1, 2, 3, 4, 5]
dual hinges:  [[0, 1], [0, 2], [0, 3], [1, 4], [1, 5], [2, 6], [2, 7], [3, 8], [3, 9], [4, 10], [6, 11], [8, 12], [10, 13], [11, 14], [12, 15]]
cubic hinges: [[0, 1], [2, 3], [4, 5], [6, 7], [7, 8], [9, 10], [10, 11], [12, 13], [13, 14], [16, 17], [23, 24], [30, 31], [38, 39], [42, 43], [46, 47]]

Drawing face 1: [0, 6, 7, 8, 1] connected by hinge 0: (0, 1)/(0, 1)
missing_atoms = [6 7 8]
Drawing face 2: [2, 9, 10, 11, 3] connected by hinge 1: (2, 3)/(0, 2)
missing_atoms = [ 9 10 11]
Drawing face 3: [5, 4, 12, 13, 14] connected by hinge 2: (4, 5)/(0, 3)
missing_atoms = [12 13 14]
Drawing face 4: [17, 18, 7, 6, 15, 16] connected by hinge 3: (6, 7)/(1, 4)
missing_atoms = [15 16 17 18]
Drawing face 5: [18, 19, 20, 21, 8, 7] connected by hinge 4: (7, 8)/(1, 5)
missing_atoms = [18 19 20 21]
Drawing face 6: [9, 22, 23, 24, 25, 10] connected by hinge 5: (9, 10)/(2, 6)
missing_atoms = [22 23 24 25]
Drawing face 7: [10, 25, 26, 27, 28, 11] connected 

In [6]:
sample_unfolding(C60,halogen=1)

KeyboardInterrupt: 